# Date Converter

We will be translating from one date format to another. In order to do this we need to connect two set of LSTMs (RNNs). The diagram looks as follows: Each set respectively sharing weights (i.e. each of the 4 green cells have the same weights and similarly with the blue cells). The first is a many to one LSTM, which summarises the question at the last hidden layer (and cell memory).

The second set (blue) is a Many to Many LSTM which has different weights to the first set of LSTMs. The input is simply the answer sentence while the output is the same sentence shifted by one. Ofcourse during testing time there are no inputs for the `answer` and is only used during training.
![seq2seq_diagram](https://i.stack.imgur.com/YjlBt.png) 

**20th January 2017 => 20th January 2009**
![troll](./images/troll_face.png)

## References:
1. https://github.com/ematvey/tensorflow-seq2seq-tutorials/blob/master/1-seq2seq.ipynb
2. The generation process was taken from: https://github.com/datalogue/keras-attention/blob/master/data/generate.py

In [ ]:
!pip install faker babel

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time

from faker import Faker
import babel
from babel.dates import format_date

import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, Embedding

import tensorflow.contrib.legacy_seq2seq as seq2seq
from utilities import show_graph

Using TensorFlow backend.


In [2]:
fake = Faker()
fake.seed(42)
random.seed(42)

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'd MMM YYY',
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY',
           ]

# change this if you want it to work with only a single language
LOCALES = babel.localedata.locale_identifiers()
LOCALES = [lang for lang in LOCALES if 'en' in str(lang)]

In [3]:
def create_date():
    """
        Creates some fake dates 
        :returns: tuple containing 
                  1. human formatted string
                  2. machine formatted string
                  3. date object.
    """
    dt = fake.date_object()

    # wrapping this in a try catch because
    # the locale 'vo' and format 'full' will fail
    try:
        human = format_date(dt,
                            format=random.choice(FORMATS),
                            locale=random.choice(LOCALES))

        case_change = random.randint(0,3) # 1/2 chance of case change
        if case_change == 1:
            human = human.upper()
        elif case_change == 2:
            human = human.lower()

        machine = dt.isoformat()
    except AttributeError as e:
        return None, None, None

    return human, machine #, dt

data = [create_date() for _ in range(50000)]

In [4]:
data[:5]

[('7 07 13', '2013-07-07'),
 ('30 JULY 1977', '1977-07-30'),
 ('Tuesday, September 14, 1971', '1971-09-14'),
 ('18 09 88', '1988-09-18'),
 ('31, Aug 1986', '1986-08-31')]

In [5]:
x = [x for x, y in data]
y = [y for x, y in data]

u_characters = set(' '.join(x))
char2numX = dict(zip(u_characters, range(len(u_characters))))

u_characters = set(' '.join(y))
char2numY = dict(zip(u_characters, range(len(u_characters))))

Pad all sequences that are shorter than the max length of the sequence

In [6]:
char2numX['<PAD>'] = len(char2numX)
num2charX = dict(zip(char2numX.values(), char2numX.keys()))
max_len = max([len(date) for date in x])

x = [[char2numX['<PAD>']]*(max_len - len(date)) +[char2numX[x_] for x_ in date] for date in x]
print(''.join([num2charX[x_] for x_ in x[4]]))
x = np.array(x)

<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>31, Aug 1986


In [7]:
char2numY['<GO>'] = len(char2numY)
num2charY = dict(zip(char2numY.values(), char2numY.keys()))

y = [[char2numY['<GO>']] + [char2numY[y_] for y_ in date] for date in y]
print(''.join([num2charY[y_] for y_ in y[4]]))
y = np.array(y)

<GO>1986-08-31


In [35]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1

In [44]:
len(char2numY)

13

In [46]:
epochs = 2
batch_size = 128
nodes = 32
embed_size = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()

inputs = tf.placeholder(tf.int32, (batch_size, x_seq_length), 'inputs')
outputs = tf.placeholder(tf.int32, (batch_size, y_seq_length), 'output')
targets = tf.placeholder(tf.int32, (batch_size, y_seq_length), 'targets')

input_embedding = tf.Variable(tf.random_uniform((len(char2numX), embed_size), -1, 1), name='enc_embedding')
output_embedding = tf.Variable(tf.random_uniform((len(char2numY), embed_size), -1, 1), name='dec_embedding')

date_input_embed = tf.nn.embedding_lookup(input_embedding, inputs)
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)
print(date_input_embed.get_shape().as_list())
# encoder
with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=date_input_embed, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    outputs, last_state = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)

print(outputs.get_shape().as_list())
#connect outputs to 
logits = tf.contrib.layers.fully_connected(outputs, num_outputs=len(char2numY), activation_fn=None) 
print(logits.get_shape().as_list())
print(targets.get_shape().as_list())
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.AdamOptimizer().minimize(loss)

[128, 29, 10]
[128, 10, 32]
[128, 10, 13]
[128, 10]


Train the graph above:

In [47]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [32]:
show_graph(tf.get_default_graph().as_graph_def())

In [48]:
sess.run(tf.global_variables_initializer())
for epoch_i in range(epochs):
    data_gen = batch_data(x, y, batch_size)
    start_time = time.time()
    for batch_i, (source_batch, target_batch) in enumerate(data_gen):
        

        loss = sess.run([loss],
            feed_dict = {inputs: source_batch,
             outputs: target_batch[:, :-1],
             targets: target_batch[:, 1:]})
        
        print('Epoch {:>3} Batch {:>4}/{} Loss: {:>6.3f}'
              .format(epoch_i, batch_i, loss))
    print('Epoch duration: '+str(time.time() - start_time))

ValueError: Cannot feed value of shape (128, 10) for Tensor 'decoding/rnn/transpose:0', which has shape '(128, 10, 32)'

In [40]:
targets.get_shape().as_list()

[128, 10]

In [ ]:
tf.nn.dynamic_rnn?

In [ ]:
tf.contrib.seq2seq.Decoder?

In [ ]:
tf.contrib.seq2seq.BasicDecoder?

In [ ]:
tf.contrib.legacy_seq2seq.

In [ ]:
tf.contrib.layers.fully_connected?

In [ ]:
tf.train.AdamOptimizer?

In [ ]:
tf.contrib.seq2seq.sequence_loss?

In [ ]:
enc_inp = [tf.placeholder(tf.int32, shape=(None, None),
           name="inp%i" % t) for t in range(x_seq_length)]

labels = [tf.placeholder(tf.int32, shape=(None, None),
            name="labels%i" % t) for t in range(y_seq_length)]

# weights = [tf.ones_like(labels_t, dtype=tf.float32)
#            for labels_t in labels]

# is_train  = tf.placeholder(tf.bool)

# # Decoder input: prepend some "GO" token and drop the final
# # token of the encoder input
# dec_inp = ([tf.zeros_like(enc_inp[0], dtype=np.int32, name="GO")] + enc_inp[:-1])

In [ ]:
memory_dim = 32
embed_dim = 32
cell = tf.contrib.rnn.BasicLSTMCell(memory_dim)
dec_outputs, dec_memory = seq2seq.embedding_rnn_seq2seq(enc_inp, dec_inp, cell, 
                                                        len(char2numX), len(char2numY), 
                                                        embed_dim,
                                                        feed_previous = tf.logical_not(is_train))

loss = seq2seq.sequence_loss(dec_outputs, labels, weights)

In [ ]:
x_seq_length

In [ ]:
y_seq_length

In [ ]:
tf.__version__